# $N$-Queens Genetic Algorithm

Initially we create a state representation, this is made up of two parts:

#### Positions. 
The positions list shows the location of each queen in a 2-Dimensional space.  It is a list of 2-tuples, $n$ long, where $n$ is the number of queens on the board.  Each tuple represents a coordinate pair $(x, y)$ for the location of a queen. For an  8-queen problem, the positions list would be represented as follows:

    [(6, 5), (6, 5), (5, 7), (7, 2), (2, 3), (6, 5), (6, 5), (1, 6)]
    
The positions are our "DNA" is this algorithm, it provides the basis from which we can generate new solutions using crossover of two parents.  To mutate the DNA we can change individual vectors within the position list.

#### Board.
The board is a 2-dimensional numpy array that is built from the position list.  This board is a boolean encoding of where all the queens are on the board, with a `1` representing a queen, and `0` an empty cell.  This is purely to make the implementation of the evaluation easier.  By taking the sum of every row, column, and diagonal, we can easily calculate the number of queens attacking each other.  A solved board would have a maximum of 1 for each row, column, and diagonal sum:

    [[1. 0. 0. 0. 0. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 1. 0.]
     [0. 0. 0. 1. 0. 0. 0. 0.]
     [0. 1. 0. 0. 0. 0. 0. 0.]
     [0. 0. 0. 0. 1. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 0. 1.]
     [0. 0. 0. 0. 0. 1. 0. 0.]
     [0. 0. 1. 0. 0. 0. 0. 0.]]

In [1]:
import numpy as np
from random import randrange, random


class State:

    def __init__(self, positions=None, board_size=8):
        '''
        Initialise a new state with random placement of queens if no positions are given,
        or generate a board with a set of 2d vectors for the location of each queen
        '''
        if positions is None:
            self.positions = []
            self.board = np.zeros((board_size, board_size))
            
            for queen in range(board_size):
                row = randrange(0, board_size)
                col = randrange(0, board_size)

                while(self.board[row, col] == 1):
                    row = randrange(0, board_size)
                    col = randrange(0, board_size)

                self.board[row, col] += 1
                self.positions.append((row, col))
                
        else:
            self.positions = positions
            self.board = np.zeros((len(positions), len(positions)))
            
            for pos in positions:
                r, c = pos
                self.board[r, c] = 1
    
    def check_rows(self) -> int:
        sigma = 0
        for row in self.board:
            if np.sum(row) > 1:
                sigma += np.sum(row)
        return int(sigma)
            
    def check_cols(self) -> int:
        sigma = 0
        for col in self.board.T:
            if np.sum(col) > 1:
                sigma += np.sum(col)
        return int(sigma)
    
    def check_diagonals(self) -> bool:
        M = self.board
        M_f = np.fliplr(M)
        n = len(M)
        
        sigma = 0
        for i in range(-n, n, 1):
            if np.sum(M.diagonal(i)) > 1:
                sigma += np.sum(M.diagonal(i))
            if np.sum(M_f.diagonal(i)) > 1:
                sigma += np.sum(M_f.diagonal(i))
                
        return int(sigma)
    
        
    def copy(self):
        '''
        Creates a deep copy of the current state
        '''
        return State(positions=self.positions.copy())

    def __str__(self) -> None:
        return str(f"{self.board}\n{self.positions}\n")



### Evaluation
To evaluate a state, we can take the sum of each row, column, and diagonal within the board.  An additional metric was added after testing, this checked that all of the tuples were only used once and if used more then once returns an extremely high evaluation (1000 in this case) to make them fatal modifications to the DNA

In [2]:
def evaluate(state: State) -> int:
    '''
    Evaluates the state of the board and returns an integer value for the number of
    faults that the board has.  A fault defined as two or more quees on the same horizontal,
    vertical or diagonal.
    '''
    s = set()
    
    for e in state.positions:
        s.add(e)
        
    if len(s) < len(state.board):
        return 1000
    
    return state.check_rows() + state.check_cols() + state.check_diagonals()


### Breeding & Crossover
To generate a child state from two parent states, we run through the list of positions of both parents concurrently.  We flip a coin with a 50% chance of inheriting a position from each parent.  If we represent each parent's DNA as a list of chars, the crossover function works as follows:
    
    parent a:  [a, b, c, d, e, f, g]
    parent b:  [A, B, C, D, E, F, G]
    
    child :    [a, B, c, d, E, F, g]
   
    
The breed function takes the entire list of parents in and generates a new child from each pair of parents.  However, in the case where two parents are the same, one parents genes are passed on directly without mutation. The matrix below shows the generation of 25 children from 5 parents.

    X |  1 |  2  |  3  |  4 |  5
    --|----|-----|-----|----|----
    1 |  1 | 12  | 13  | 14 | 15
    --|----|-----|-----|----|----
    2 | 21 |  2  | 23  | 24 | 25
    --|----|-----|-----|----|----
    3 | 31 | 32  |  3  | 34 | 35
    --|----|-----|-----|----|----
    4 | 41 | 42  | 43  |  4 | 45
    --|----|-----|-----|----|----
    5 | 51 | 52  | 53  | 54 |  5

In [3]:
def breed(parents: list) -> list:
    '''
    Take a population and breed every combination of parents to create children
    The output will be a list of n^2 children, where n is the original number of parents
    '''
    children = []
    
    for i in range(len(parents)):
        
        for j in range(len(parents)):
            
            if i == j:
                children.append(parents[i])
            
            else:
                children.append(mutate(crossover(parents[i], parents[j])))
                
    return children

In [4]:
def crossover(parent_a: State, parent_b: State) -> State:
    '''
    Create a new state that inherits about 50% of it's atributes from each parent
    '''
    child = []
    for i in range(len(parent_a.positions)):
        
        if random() > 0.5:
            child.append(parent_a.positions[i])
        else:
            child.append(parent_b.positions[i])
    
    return State(child)
        

In [5]:
alpha = 0.1

def mutate(state: State) -> State:
    '''
    Return a new state that is a mutated version of the state passed in, where mutation is defined 
    as a random change in any integer in the positions (or "DNA") of the state 
    '''
    n = len(state.board)
    positions = state.positions.copy() 
    
    for i in range(len(positions)):
        
        if random() < alpha:
            
            r = randrange(1, n)
            c = randrange(1, n)
            
            while(state.board[r, c] == 1):  # Ensure we aren't overwriting another queen
                r = randrange(1, n)
                c = randrange(1, n)
                
            positions[i] = (r, c)
    
    return State(positions=positions)
    

In [6]:
def select(gen: list) -> list:
    '''
    Select the top 10% of the population, with the lowest evaluation function being the "fittest"
    '''
    gen.sort(key=evaluate)
    n = int(len(gen) / 10)
    return gen[:n]
    

In [12]:
from time import time

gen = [State(board_size=4) for i in range(100)]
best = 1000
count = 1

while best != 0 and count < 200:
    start = time()
    gen = select(gen)
    best = evaluate(gen[0])    
    print(gen[0])
    gen = breed(gen)
    
    print(f"Gen {count}: {best} in {time() - start}ms")
    count += 1
    
print("\nSolution:")
print(gen[0])

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
[(0, 0), (3, 0), (1, 1), (2, 3)]

Gen 1: 4 in 0.014250755310058594ms
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[(1, 0), (3, 1), (2, 3), (0, 3)]

Gen 2: 2 in 0.006580829620361328ms
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[(1, 0), (3, 1), (2, 3), (0, 3)]

Gen 3: 2 in 0.009377002716064453ms
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[(1, 0), (0, 2), (2, 3), (3, 1)]

Gen 4: 0 in 0.007125377655029297ms

Solution:
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[(1, 0), (0, 2), (2, 3), (3, 1)]

